In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import pandas as pd
import numpy as np


c:\Users\vishe\miniconda3\envs\new_ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import time

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

In [5]:
df1 = pd.read_csv(r'C:\Kaam\naukri\archive\articles1.csv')
df2 = pd.read_csv(r'C:\Kaam\naukri\archive\articles2.csv')
df3 = pd.read_csv(r'C:\Kaam\naukri\archive\articles3.csv')

In [6]:
dff = pd.concat([df1,df2,df3]).sample(frac=1).reset_index(drop=True)

In [7]:
dff

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,75404,115406,An Alternative to Increasing the Minimum Wage,National Review,Pascal-Emmanuel Gobry,2017-02-16,2017.0,2.0,http://www.nationalreview.com/article/444982/m...,If there’s one blessing in the election of Don...
1,125577,187608,"Syrian army, with Russian air support, advance...",Reuters,Dominic Evans,2016-03-26,2016.0,3.0,http://www.reuters.com/article/us-mideast-cris...,Syrian government forces advanced into Palmyr...
2,108689,162436,"Bill Paxton, Prolific Actor And Star Of ’Titan...",NPR,Colin Dwyer,2017-02-26,2017.0,2.0,http://www.npr.org/sections/thetwo-way/2017/02...,"Updated at 12:38 p. m. ET, Bill Paxton, prolif..."
3,111556,166090,LISTEN: Jeb Bush Says Trump Is ’A Creature Of ...,NPR,NaN,2015-12-31,2015.0,12.0,https://web.archive.org/web/20160101025934/htt...,Republican presidential candidate Jeb Bush pro...
4,85659,128413,These sex addicts can’t stop swiping right on ...,New York Post,Melkorka Licea,2017-04-02,2017.0,4.0,https://web.archive.org/web/20170403002211/htt...,"Cindy, a Bushwick graphic designer, became ..."
...,...,...,...,...,...,...,...,...,...,...
142565,135316,204530,Bravado and branding: Trump brings a new leade...,Washington Post,Marc Fisher,2017-01-19,2017.0,1.0,https://web.archive.org/web/20170120005827/htt...,From his picture window overlooking New ...
142566,51441,71281,Colin Kaepernick protests national anthem and ...,Business Insider,Cork Gaines,2016-08-28,2016.0,8.0,NaN,’ ’ ” Colin Kaepernick caused a stir on Fri...
142567,42120,61062,Say cheese! Hubble telescope spots Smiley face...,CNN,NaN,2015-02-10,2015.0,2.0,NaN,"(CNN) Look! Up in the sky, it’s a bird, it’s ..."
142568,92982,138113,Is Calvin Harris’ new single about Taylor Swift?,New York Post,Hardeep Phull,2016-04-29,2016.0,4.0,http://nypost.com/2016/04/29/is-calvin-harris-...,If Rihanna’s experimental latest album left ...


In [8]:
# shuffling_factor = 0.8
# numrows = 10000
# a, b = numrows//3, numrows//4

# r1 = df1.sample(a)
# r2 = df2.sample(b)
# r3 = df3.sample(numrows-a-b)


# dff = pd.concat([r1,r2,r3]).sample(frac=1).reset_index(drop=True)
dff2 = dff[['id', 'title', 'content']]
# dff2.to_string()
mask2 = dff2['title'].str.contains('nan', na = False)
exclude_strings = ["stopwords", "another", 'nan','trump', 'Trump','government', 'russia', 'president', 'hillary']

dff2 = dff2[~dff2['content'].str.contains('|'.join(exclude_strings), case=False)]

dff2 = dff2[~mask2]
dff2

C:\Users\vishe\AppData\Local\Temp\ipykernel_21084\3590037805.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dff2 = dff2[~mask2]


,id,title,content
2,162436,"Bill Paxton, Prolific Actor And Star Of ’Titan...","Updated at 12:38 p. m. ET, Bill Paxton, prolif..."
10,62552,Photographer unites world’s greatest rock stars,"The Hague, Netherlands (CNN) It’s possibly the..."
11,124848,Woman dies after getting arm stuck in donation...,"NATALIE, Pa. — A Pennsylvania woman has die..."
12,42073,Fox Searchlight to Register Voters Before ’Bir...,Fox Searchlight has teamed with several indepe...
16,140788,Family finds dog that’s been missing for a decade,"BIRMINGHAM, Ala. — Tracy Dove assumed the w..."
...,...,...,...
142557,50362,3 dead in Texas plane crash collision,(CNN) Two small planes collided in Texas on S...
142558,29513,The Combat of ’Horizon Zero Dawn’ - Trailer - ...,The latest trailer for Horizon Zero Dawn explo...
142566,71281,Colin Kaepernick protests national anthem and ...,’ ’ ” Colin Kaepernick caused a stir on Fri...
142567,61062,Say cheese! Hubble telescope spots Smiley face...,"(CNN) Look! Up in the sky, it’s a bird, it’s ..."


In [9]:
# Download NLTK stopwords (if not already downloaded)
# nltk.download('stopwords')
# nltk.download('punkt')

# Assuming 'texts' is a list of text documents

# Get English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Remove stopwords from each document
texts_without_stopwords = []
for text in dff2['content']:
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    texts_without_stopwords.append(" ".join(filtered_text))
dff2['content']=texts_without_stopwords

In [10]:
# Step 1: Vectorize the text data
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(list(dff2['content']))

# Step 2: Apply dimensionality reduction (optional)
# You may want to reduce dimensionality to improve clustering performance
svd = TruncatedSVD(n_components=100)
X = svd.fit_transform(X)

# Step 3: Cluster the data using K-means
n_clusters = 8  # You can adjust this number based on your dataset
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_

# Step 4: Output cluster labels
# You can use these labels to analyze or categorize your data
print("Cluster labels:", cluster_labels)

# Step 5: Optionally, assign cluster labels to each document
# For a large dataset, you may want to output the cluster assignments to a file
# Here, I'm just printing the cluster assignments for the first few documents
for i in range(min(10, len(dff2))):
    print("Document:", i, "Cluster:", cluster_labels[i])


Cluster labels: [5 5 3 ... 2 3 3]
Document: 0 Cluster: 5
Document: 1 Cluster: 5
Document: 2 Cluster: 3
Document: 3 Cluster: 3
Document: 4 Cluster: 3
Document: 5 Cluster: 3
Document: 6 Cluster: 4
Document: 7 Cluster: 4
Document: 8 Cluster: 0
Document: 9 Cluster: 3


In [11]:
dff2['clusters']=cluster_labels

In [19]:
dff2[dff2['clusters']==7]

,id,title,content,clusters
77,172192,"PHOTOS: Model London Set Afloat, Then Aflame, ...","weekend , London burning — . buildings harmed ...",7
145,54312,"Plane collides with truck at LAX, injuring 8",( CNN ) passenger jet taxiing gate Los Angeles...,7
146,56374,"Jupiter’s giant, red hot storm heats the entir...","( CNN ) Jupiter ’ Great Red Spot , humungous s...",7
185,169429,Rising Seas Push Too Much Salt Into The Florid...,Florida Everglades swampy wilderness size Dela...,7
217,79669,Rocked By Turbulence,"us continue writing great stories , need displ...",7
...,...,...,...,...
142441,145511,Teens rescued from capsized boat miles from shore,Dramatic photos released cops Florida show thr...,7
142457,101074,The California Desert Is Experiencing A Rare ...,’ rare “ super bloom ” wildflowers happening r...,7
142483,172823,U.S. Targets Chinese Company For Supporting N....,U. S. targeting Chinese company people run all...,7
142516,142733,"US cargo plane with dead body, millions in cas...","HARARE , Zimbabwe — Zimbabwean aviation author...",7


In [20]:
wanted_labels = [6]
dff3 = dff2.loc[dff2['clusters'].isin(wanted_labels)]

In [21]:
dff=dff[dff['id'].isin(dff3['id'])]
dff=dff.drop(['url'], axis=1)
dff = dff[['id', 'title', 'content']]

In [22]:
dff

,id,title,content
34,134713,Caesars owners talked IPO while dissing shares,Caesars Entertainment’s backers hoped to spr...
46,192008,Exclusive: Dr Pepper Snapple in talks to buy B...,The talks come after Reuters reported on Oct. ...
75,127407,Playboy ditching its oldest subscribers in bid...,Playboy owner Rizvi Traverse has abandoned its...
121,72270,"Wish, the $3B startup challenging Amazon, is o...","’ ’ ” Wish, the stealthy company that’s c..."
136,110602,Samsung Lost An Entire HP In Market Value Aft...,Samsung’s disastrously defective new Galaxy ...
...,...,...,...
142281,65310,Tesla’s new Powerpack battery storage project ...,"’’ ’MIRA LOMA, California — Tesla cut the r..."
142359,186262,"Earnings bolster Cisco, but raise concerns for...",Networking giant Cisco Systems Inc reported i...
142362,68015,Yahoo just admitted that it overpaid for Tumblr,’’ ’Yahoo CEO Marissa Mayer promised the hip ...
142382,21602,"Apple Moves Toward a Wireless Future, One Twea...",SAN FRANCISCO — During a product showcase...


In [23]:
device = torch.device("cuda")
print(device)


cuda


In [24]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)

In [25]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=1000, # Rank
    lora_alpha=500,
    target_modules=["q", "v"],
    lora_dropout=0.0005,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [26]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [27]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 294912000
all model parameters: 1078062080
percentage of trainable model parameters: 27.36%


In [28]:
training = dff.sample(frac=0.5).reset_index(drop=True)
testing = dff.sample(frac=0.35).reset_index(drop=True)
validation = dff.sample(frac=0.15).reset_index(drop=True)

In [29]:
from datasets import Dataset, DatasetDict

# Example DataFrames (replace with your actual data)
train_df = training
validation_df = validation
test_df = testing

# Create datasets from DataFrames
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

# Access individual splits
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'content'],
        num_rows: 1310
    })
    validation: Dataset({
        features: ['id', 'title', 'content'],
        num_rows: 393
    })
    test: Dataset({
        features: ['id', 'title', 'content'],
        num_rows: 917
    })
})


In [30]:
data = dataset_dict

In [31]:
def tokenize_function(example):
    start_prompt = 'Provide a Subject for the given business email content:\n\n'
    end_prompt = '\n\nTitle: '
    prompt = [start_prompt + content + end_prompt for content in example["content"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["title"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = data.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'title', 'content'])

Map: 100%|██████████| 917/917 [00:00<00:00, 975.65 examples/s]


In [32]:
output_dir = f'./peft-content-title-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1    
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],

)

c:\Users\vishe\miniconda3\envs\new_ml\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [33]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (1310, 2)
Validation: (393, 2)
Test: (917, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1310
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 393
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 917
    })
})


In [34]:
peft_trainer.train()

peft_model_path="./peft-content-title-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

  0%|          | 0/1 [00:19<?, ?it/s]

                                          
100%|██████████| 1/1 [00:26<00:00, 26.14s/it]


{'loss': 40.2411, 'grad_norm': 19.766342163085938, 'learning_rate': 0.0, 'epoch': 0.0}
{'train_runtime': 26.1452, 'train_samples_per_second': 0.038, 'train_steps_per_second': 0.038, 'train_loss': 40.241092681884766, 'epoch': 0.0}


('./peft-content-title-checkpoint-local\\tokenizer_config.json',
 './peft-content-title-checkpoint-local\\special_tokens_map.json',
 './peft-content-title-checkpoint-local\\spiece.model',
 './peft-content-title-checkpoint-local\\added_tokens.json',
 './peft-content-title-checkpoint-local\\tokenizer.json')

In [35]:
from peft import PeftConfig, PeftModel

adapter_model_name = r"C:\Kaam\naukri\peft-content-title-checkpoint-local"

peft_model = PeftModel.from_pretrained(original_model, adapter_model_name)



In [ ]:
torch.cuda.empty_cache()

In [36]:
content = data['test'][:100]['content']
human_baseline_summaries = data['test'][:100]['title']

original_model_summaries = []
peft_model_summaries = []

for idx, content in enumerate(content):
    prompt = f"""Provide a Subject for the given business email content:
{content}
Title:
"""
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=600))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    human_baseline_text_output = human_baseline_summaries[idx]
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=600, do_sample = True, top_k = 20, top_p = 0.31))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)
    print(human_baseline_summaries[idx])
    print(original_model_text_output)
    print(peft_model_text_output)


Token indices sequence length is longer than the specified maximum sequence length for this model (1222 > 512). Running this sequence through the model will result in indexing errors


Apple CEO blames iPhone 8 rumors for slow iPhone sales
Apple's iPhone Sales Dropped Last Quarter
Apple's iPhone Sales Dropped in Quarter
How to tell if your iPhone suffers from so-called ’Touch Disease,’ and what to do next
What You Need to Know About 'Touch Disease'
How to Fix iPhone 6 Plus 'Touch Disease'
Uber and Lyft are demolishing New York City taxi drivers
New York Taxi Drivers Are Taking a Hit
Taxi Rides in NYC
This unmarked van looks like it could be an Apple self-driving car or mapping vehicle
Apple Maps Car
Apple Maps Car
Impatient with Yahoo, some investors cheer Starboard effort
Yahoo's Board of Directors Campaign Gets Support from Investors
Investors Welcome Starboard’s Campaign to Replace Yahoo’s Board
Starboard’s Smith says Yahoo ’welcoming’ since his arrival
Yahoo welcomes Starboard Value LP CEO
Yahoo's Board Welcomes Starboard Value LP CEO
Exclusive: Keysight Technologies explores acquisition of Ixia: sources
Keysight Technologies to bid for Ixia
Keysight to bid for I

In [37]:
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries',  'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Apple CEO blames iPhone 8 rumors for slow iPho...,Apple's iPhone Sales Dropped Last Quarter,Apple's iPhone Sales Dropped in Quarter
1,How to tell if your iPhone suffers from so-cal...,What You Need to Know About 'Touch Disease',How to Fix iPhone 6 Plus 'Touch Disease'
2,Uber and Lyft are demolishing New York City ta...,New York Taxi Drivers Are Taking a Hit,Taxi Rides in NYC
3,This unmarked van looks like it could be an Ap...,Apple Maps Car,Apple Maps Car
4,"Impatient with Yahoo, some investors cheer Sta...",Yahoo's Board of Directors Campaign Gets Suppo...,Investors Welcome Starboard’s Campaign to Repl...
...,...,...,...
95,Herbalife Settlement With F.T.C. Ends Billiona...,The Herbalife Settlement,Herbalife’s Settlement with the F. T. C.
96,Uber used secret tool to deceive authorities: ...,Uber reportedly used'sting' to evade authorities,Uber reportedly used'sting' to evade authorities
97,Art sales aren’t booming despite record sell-off,"The art market is healthy, but not a peak","The art market is healthy, but not peaked"
98,Wells Fargo CEO says bank changing handling of...,Wells Fargo CEO: ‘It’s not about the money’,Wells Fargo CEO to Address Employee Concerns
